In [ ]:
#reciver
import cv2
import socket
import pickle
import numpy as np

s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
ip = "127.0.0.1"
port = 6666
s.bind((ip, port))

while True:
    x = s.recvfrom(1000000)
    clientip = x[1][0]
    data = x[0]

    data = pickle.loads(data)
    
    img = cv2.imdecode(data, cv2.IMREAD_COLOR)
    
    cv2.imshow('Img Server', img)

    if cv2.waitKey(5) & 0xFF == 27:
        break

cv2.destroyAllWindows()

In [ ]:
import cv2
import socket
import pickle
import numpy as np
import mediapipe as mp

s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
s.setsockopt(socket.SOL_SOCKET, socket.SO_SNDBUF, 1000000)

server_ip = "127.0.0.1"
server_port = 9999

cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True)

overlay_name = "glitter.png"
overlay_img = cv2.imread(overlay_name, -1)

overlay_img = cv2.resize(overlay_img, (100, 100))

current_pos = 2 
print("press 1, 2, 3, or q")

while cap.isOpened():
    ret, img = cap.read()
    if not ret: break

    img = cv2.flip(img, 1)
    rgb_frame = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)

    key = cv2.waitKey(5) & 0xFF
    
    if key == ord('1'): 
        current_pos = 1
    elif key == ord('2'): 
        current_pos = 2
    elif key == ord('3'): 
        current_pos = 3
    elif key == ord('q'): 
        break

    if results.multi_face_landmarks:
        for landmarks in results.multi_face_landmarks:
            h, w, _ = img.shape

            if current_pos == 1:
                point = landmarks.landmark[10] # Forehead
            elif current_pos == 2:
                point = landmarks.landmark[4]  # Nose
            else:
                point = landmarks.landmark[152] # Moth

            cx, cy = int(point.x * w), int(point.y * h)

            ih, iw = overlay_img.shape[:2]
            y1, y2 = cy - ih//2, cy + ih//2
            x1, x2 = cx - iw//2, cx + iw//2

            y1_clamped = max(0, y1)
            y2_clamped = min(h, y2)
            x1_clamped = max(0, x1)
            x2_clamped = min(w, x2)

            if y1_clamped < y2_clamped and x1_clamped < x2_clamped:
                ov_y1 = y1_clamped - y1
                ov_y2 = ov_y1 + (y2_clamped - y1_clamped)
                ov_x1 = x1_clamped - x1
                ov_x2 = ov_x1 + (x2_clamped - x1_clamped)

                img[y1_clamped:y2_clamped, x1_clamped:x2_clamped] = \
                    overlay_img[ov_y1:ov_y2, ov_x1:ov_x2]

    cv2.imshow('Img Client', img)

    ret, buffer = cv2.imencode(".jpg", img, [int(cv2.IMWRITE_JPEG_QUALITY), 30])
    x_as_bytes = pickle.dumps(buffer)
    
    s.sendto(x_as_bytes, (server_ip, server_port))

cv2.destroyAllWindows()
cap.release()